<p style="text-align: center;font-size: 40pt">Use case 2</p>

\section{Power Plant Inspection}\label{sec:inspection}
%=========================================================

% Describe the application
In collaboration with \acx{air}, prototypes were developed for the inspection and maintenance of industrial plants.
Some inspection tasks need to move inspection tools in environments that are difficult to access by human due to dimensional, temperature or air quality constraints.
The use of mobile systems for inspection can not only deal with those constraints, but also can reduce the time and costs of inspections.
This would, for example, allow for the inspection of critical pieces of equipment on location, without the need to dismantle any structures. Similarly, the installation of scaffolding around a structure becomes unnecessary, thus saving inspection time.
The typical environments encountered during inspection procedures are confined spaces (indoors) with well structured, static surfaces.


\begin{figure}[htb]
\centering
	\includegraphics[width=0.8\textwidth]{robot_magnebike}
	\caption{Three prototypes of chest inspection robots without the sensors, in a mock-up of a steam chest.
	The sensor was not installed at the time of the photograph.}
	\label{fig:robotMagnebike}
\end{figure}

% Describe the platform
For the specific task of steam chest inspection, a robot was developed with high mobility capabilities and a compact size \citep{Tache:2009uy}.
The robot, named Magnebike, moved around a metallic, cylindric environment by using its two magnetic wheels positioned in the same configuration as a bike (\autoref{fig:robotMagnebike}).
Those specialized wheels coupled with small lever arms allowed the platform to move up-side-down, pass 90$^\circ$ edges and navigate in high curvature tube.
A considerable amount of effort were invested in reducing the size of the platform, which ultimately gave a small volume of 0.006\,m$^3$ and a weight of 0.34\,kg.
The platform was the slowest presented in this section, with a maximal velocity value of only 0.045 m/s.
The robot gathered high-resolution scans (340,000 points) with a refresh time of 50\,s.
The 3D scans were assembled from a 2D Hokuyo URG-04LX.
Pre-alignment of the scans were ensured by wheel odometry, which displays virtually no slip because of the magnetic force hold the wheels on the surfaces.
An \acx{imu} was used in conjunction with the odometry to cope with the 3D nature of the motion \citep{Tache:2011ef}.
The main sources of pre-alignment errors were: (1) motions perpendicular to the gravity vector that were not observable with the sensor used, and (2) the rapid cumulation of errors by the low-cost \acx{imu} used on the slow-moving robot.

%--------------------------------------------------
%\subsection{Steam Chest Reconstruction}
% Describe the solution
During inspection, the robot is intended to be tethered for safety reason, which solved the communication problem between the embedded computer and a faster system used as control station.
Because the system must not damage the inspected structure in any case, it traveled slowly in the environment, reducing the pressure on the real-time requirement for the registration.
However, the operator might not have a visual contact with the robot at all times during inspection.
Therefore, the map resolution must be high enough to detect obstacles and holes during remote operations.
The number of points produced by the scanner was significantly larger than strictly necessary for a proper registration.
To reduce rapidly this number of points, we randomly removed 90\,\% of the point as soon as the scan were recorded.
We used a maximal density of 20000 points per m$^3$ to cope with the small size of inspected environments.
For the registration, we did not use any pre-alignment to test the worst case scenario (i.e., when the rotation is not observable by the \acx{imu}).
This forced us to extend the maximal matching distance to 0.5\,m, even if a scan was taken roughly at every 0.30\,m.
Given that scans are taken by an operator at a fix and short interval, we used an outlier ratio of 80\,\%.
The complete list of modules used with their main parameters are listed in \autoref{tab:icpConfigAppMagnebike}.


\begin{table}[htp]
\caption{Configuration of the \icp chain for the Magnebike mapping applications.
The definition of the column \emph{Step} follows \autoref{chap:icpSolutions}. 
The names used in the column \emph{Module} refer to specific implementation documented in the open source library \texttt{libpointmatcher}.
}
\label{tab:icpConfigAppMagnebike}
\centering
\footnotesize
\begin{tabularx}{\textwidth}{l l X}

\toprule
\emph{Step} & \emph{Module} & \emph{Description} \\
\midrule
DF\textsubscript{read}
  & \texttt{RandomSampling} & Keep randomly 10\,\% of the points. \\
  & \texttt{SamplingSurfaceNormal} & Keep 80\,\% of the points, while extracting surface normals based on 20 \acx{nn}. \\
  & \texttt{ObservationDirection} & Add vector pointing toward the origin of the sensor.\\
  & \texttt{OrientNormals} & Orient surface normals toward the observation direction.\\
  & \texttt{MaxDensity} & Subsample to keep point with density of 20000\,pts/m$^3$.\\
\midrule
DF\textsubscript{ref}
  & \texttt{SurfaceNormal} & Compute normal and density with  20 \acx{nn} and an approximation factor $\epsilon=3.16$. \\
  & \texttt{MaxDensity} & Subsample to keep point with density of 20000\,pts/m$^3$.\\
\midrule
MF 
  & \texttt{KDTree} & Use an approximate kD-tree with a maximum matching distance of 0.5\,m and an approximation factor of $\epsilon=3.16$.\\
\midrule
OF
  & \texttt{TrimmedDist} & Keep 80\,\% closest paired points. \\
  & \texttt{SurfaceNormal} & Remove paired points with normals angle larger than 50$^\circ$.\\
\midrule
EM
  & \texttt{PointToPlane} & Objective function using point-to-plane \mbox{error}. \\
\midrule
TC 
  & \texttt{Differential} & Stop after a minimum error below 0.01\,m and 0.001\,rad. \\
  & \texttt{Counter} & Stop after the iteration count reached 100. \\
  & \texttt{Bound} & Stop if transformation increases beyond 5.0\,m and 0.8\,rad. \\
\bottomrule

\end{tabularx}

\raggedright
\scriptsize
\emph{Legend}:\\ DF\textsubscript{read} = Data Filters for readings, DF\textsubscript{ref} = Data Filters for references, MF = Matching Function, OF = Outlier Filters, EM = Error Minimizer, TC = Transformation Checker.
\end{table}

% Describe 3D results
To test the mapping capability of the platform, a real steam chest was made available by \acx{air}. This part was actually removed from a power plant for reparation purpose.
Multiple inspection runs were executed, each run starting from one of the seven entry points (\autoref{fig:resultsSteamChest}).
We only present here the results from the longest path since it covered the entire environment.
The robot started on one side of the steam chest, situated on the left of \autoref{fig:resultsSteamChest}.
Each 3D scan was taken on a stop-and-go strategy at every 0.1\,m.
The total path covered a distance of 5.8\,m for a total of 59 scans.
All runs were registered offline approximately 10 times faster than the rate at which they were recorded.
Based on this reconstruction, it was possible to have full 3D path planning and navigation in this environment~\citep{Stumm:2012jp}.

%We thank Fabien T\^{a}che and Andreas Breitenmoser (\acx{asl}) for providing the recorded point clouds.


\begin{figure}[htbp]
\centering
	\includegraphics[width=0.88\textwidth]{steamChestMap}
	\includegraphics[width=0.58\textwidth]{steamChest}
\caption{Deployment results of Magnebike in a real steam chest.
\emph{Top}: Cut view of the reconstructed environment. 
The light gray line correspond to the path of the robot, with the sphere being the positions where the robot stopped to take a 3D scan.
The colors of the map follow the discreet time (from 0 to 58) at which the scans where taken.
\emph{Bottom}: The actual steam chest removed for maintenance.
}
\label{fig:resultsSteamChest}
\end{figure}